In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
pwd

'c:\\Users\\ubaid\\Downloads\\MLOps\\SalesForecasting'

In [1]:
# included it in src\videogamesforecasting\entity\config_entity.py

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    dir: Path
    data_dir: Path
    STATUS_FILE: str
    all_schema: dict



In [2]:
import os
from videogamesforecasting.constants import *
from videogamesforecasting.utils.common import read_yaml, create_directories
from videogamesforecasting.entity.config_entity import DataValidationConfig

class ConfigurationManager:
    def __init__(
        self,
        # from constants
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        (create_directories([self.config.artifacts_root]))

    # included it in src\videogamesforecasting\config\configuration.py

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([os.path.split(config.STATUS_FILE)[0]])

        get_data_validation_config = DataValidationConfig(
            dir=config.dir,
            data_dir=config.data_dir,
            STATUS_FILE=config.STATUS_FILE,
            all_schema=schema
        )

        return get_data_validation_config

In [ ]:
# included it in src\videogamesforecasting\components\data_validation.py

from videogamesforecasting.logging import logger
import pandas as pd
import yaml

class DataValidation:
    def __init__(self, config_path="config/config.yaml",schema_filepath="schema.yaml"):
        with open(config_path, "r") as config_file:
            self.config = yaml.safe_load(config_file)["data_validation"]
        with open(schema_filepath,"r") as schema_file:
            self.schema=yaml.safe_load(schema_file)["COLUMNS"]

        self.source_config = self.config
        self.schema_config=self.schema.keys()

       
    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.source_config['data_dir'])
            all_cols = list(data.columns)

            all_schema = self.schema_config

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.source_config['STATUS_FILE'], 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.source_config['STATUS_FILE'], 'w') as f:
                        f.write(f"Validation Status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [ ]:
try:
    config=ConfigurationManager()
    config.data_validation_config()
    validate=DataValidation()
    validate.validate_all_columns()
except Exception as e:
    raise e